In [1]:
# NiN是network in network，提出的想法影响了后续的网络设计
# 卷积的输入输出是4D矩阵，而全连接是2D矩阵。如果直接从卷积
# 转换到全连接，会导致全连接层有过多的参数。而NiN的想法是，
# 只对通道层做全连接，并且像素之间通过共享权重来解决这个问题
# 即，使用kernel大小为1×1，进行卷积操作


from mxnet.gluon import nn


def mlpconv(channels, kernel_size, padding, strides=1, max_pooling=True):
    out = nn.Sequential()
    with out.name_scope():
        out.add(nn.Conv2D(channels=channels, kernel_size=kernel_size,
                          strides=strides, padding=padding, activation='relu'))
        out.add(nn.Conv2D(channels=channels, kernel_size=1, padding=0,
                          strides=strides, activation='relu'))
        out.add(nn.Conv2D(channels=channels, kernel_size=1, padding=0,
                          strides=strides, activation='relu'))
        if max_pooling:
            out.add(nn.MaxPool2D(pool_size=3, strides=2))
    return out

In [2]:
from mxnet import nd

blk = mlpconv(64, 3, 0)
blk.initialize()

x = nd.random_uniform(shape=(32, 3, 16, 16))
y = blk(x)
print(y.shape)

(32, 64, 6, 6)


In [3]:
# 除了使用1×1的卷积之外，NIN在最后不再使用全连接，而是使用通道数作为输出类别个数
# 外接一个平均池化层来将每个通道的数值转换成标量

net = nn.Sequential()
with net.name_scope():
    net.add(mlpconv(96, 11, 0, strides=4))
    net.add(mlpconv(256, 5, 2))
    net.add(mlpconv(384, 3, 1))
    net.add(nn.Dropout(.5))
    net.add(mlpconv(10, 3, 1, max_pooling=False))
    net.add(nn.AvgPool2D(pool_size=5)),
    net.add(nn.Flatten())

In [4]:
import utils
from mxnet import gluon
from mxnet import init

In [5]:
train_data, test_data = utils.load_data_fashion_mnist_new(batch_size=64, resize=224)

MXNetError: [13:28:44] c:\jenkins\workspace\mxnet-tag\mxnet\src\storage\./cpu_device_storage.h:72: Failed to allocate CPU Memory

In [ ]:
ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
utils.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=1)